In [1]:
# !pip install openai

In [2]:
# !pip install --upgrade openai

In [19]:
import os
import openai
import time
import pandas as pd

In [1]:
openai.organization = ""#"X"
openai.api_key = "X"
openai.Model.list()

NameError: name 'openai' is not defined

In [21]:
test_sample = pd.read_excel("D:/github/abstract-prediction/data_test.xlsx")

In [40]:
template = lambda t, a: f"""Title: {t}
Abstract: {a}
Based on the abstract/title, Is the abstract INCLUDE or EXCLUDE in our literature review? Think step by step."""
t = test_sample["Article Title"].iloc[0] 
a = test_sample["Abstract (AB)"].iloc[0]
prompt = template(t, a)
prompt

'Title: Does certified organic farming reduce greenhouse gas emissions from agricultural production?\nAbstract: The increasing prevalence of ecologically sustainable products in consumer markets, such as organic produce, are generally assumed to curtail anthropogenic impacts on the environment. Here I intend to present an alternative perspective on sustainable production by interpreting the relationship between recent rises in organic agriculture and greenhouse gas emissions from agricultural production. I construct two time series fixed-effects panel regressions to estimate how increases in organic farmland impact greenhouse gas emissions derived from agricultural production. My analysis finds that the rise of certified organic production in the United States is not correlated with declines in greenhouse gas emissions derived specifically from agricultural production, and on the contrary is associated positively overall agricultural greenhouse gas emissions. To make sense of this find

In [41]:
def chatgpt(message, text, temp=0.7, max_t=5):
    try: 
        output = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                  {"role": "system", "content": message},
                  {"role": "user", "content": text},
            ],
            temperature=temp,
            max_tokens=max_t,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
    except:
        print("Thinking...")
        time.sleep(1)
        return chatgpt(message, text, temp=temp, max_t=max_t)
    return output

In [42]:
%%time
system = "You are an expert on agroecology and impact on climate change. You have to decide which article is include or excluded in our literature review. "
chatgpt(system, prompt, temp=0.7, max_t=50)["choices"][0]["message"]["content"]

Wall time: 1.81 s


'Based on the abstract, the article should be INCLUDED in the literature review. The article presents an alternative perspective on sustainable production and examines the relationship between organic farming and greenhouse gas emissions, which is directly related to the topic of agroecology and impact'

In [43]:
t, a, label = test_sample.sample(1).iloc[0]["Article Title  Abstract (AB)  Final decision".split("  ")]
prompt = template(t, a)
prompt, label

("Title: Importance of Soils of Agroecosystems for Climate Change Policy\nAbstract: Until now, international climate change policy has neglected the potential of agroecosystems to contribute to climate change mitigation by reduction in net greenhouse gas (GHG) emissions, and by enhancing soil inorganic carbon (SIC) and soil organic carbon (SOC) sequestration. Proven agricultural practices for creating a positive soil C balance such as those under the umbrella of climate-smart agriculture (CSA) are readily available. In the future, the importance of soil C stocks for climate change mitigation will be addressed by the target of Land Degradation Neutrality (LDN) of United Nations Convention to Combat Desertification (UNCCD), and land- and soil-related targets for achieving United Nations Sustainable Development Goals (SDGs). Further, many nationally determined contributions (NDCs) submitted to United Nations Framework Convention on Climate Change (UNFCCC) after the 2015 Paris Agreement in

In [44]:
chatgpt(system, prompt, temp=0.7, max_t=50)["choices"][0]["message"]["content"]

'The abstract is INCLUDE in our literature review as it discusses the importance of agroecosystems in contributing to climate change mitigation and enhancing soil carbon sequestration. It also highlights the need for better soil management practices and policies that address the importance of'

In [45]:
%%time
responses = []
for i, ix in enumerate(test_sample.index):
    t, a, label = test_sample.loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    o = chatgpt(system, template(t, a), temp=0.7, max_t=50)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/test_sample.shape[0], ix, o, label)

0.25 0 Based on the abstract, the article should be included in the literature review as it presents an alternative perspective on the relationship between organic farming and greenhouse gas emissions from agricultural production. The article uses regression analysis to study the impact of organic farming on greenhouse gas emissions INCLUDE
0.5 1 Based on the title and abstract, the article is potentially relevant to our literature review as it discusses the simulation of N2O emissions from dairy farms and strategies to reduce greenhouse gas emissions. Therefore, the abstract is INCLUDED in our literature review. EXCLUDE
0.75 2 Step 1: The article's title is "Predicting and mitigating the net greenhouse gas emissions of crop rotations in Western Europe."  Step 2: The abstract explains how the authors used experimental data to calibrate and evaluate a biophysical crop model INCLUDE
1.0 3 Step 1: The article is about greenhouse gas emissions from nitrogen fertilizer use in China.  Step 2

In [46]:
test_pred = test_sample.copy()
test_pred["chatgpt"] = [x[1] for x in responses]
test_pred

,Unnamed: 0,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
0,1,J,"McGee, JA",Does certified organic farming reduce greenhou...,The increasing prevalence of ecologically sust...,2015.0,10.1007/s10460-014-9543-1,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN,"Based on the abstract, the article should be i..."
1,3,J,"Chianese, DS; Rotz, CA; Richard, TL",SIMULATION OF NITROUS OXIDE EMISSIONS FROM DAI...,Farming practices can have a large impact on t...,2009.0,NaN,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,"Based on the title and abstract, the article i..."
2,21,J,"Lehuger, S; Gabrielle, B; Laville, P; Lamboni,...",Predicting and mitigating the net greenhouse g...,"Nitrous oxide, carbon dioxide and methane are ...",2011.0,10.1016/j.agrformet.2011.07.002,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN,"Step 1: The article's title is ""Predicting and..."
3,27,J,"Kahrl, F; Li, YJ; Su, YF; Tennigkeit, T; Wilke...",Greenhouse gas emissions from nitrogen fertili...,The use of synthetic nitrogen (N) fertilizers ...,2010.0,10.1016/j.envsci.2010.07.006,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Step 1: The article is about greenhouse gas em...
4,31,J,"Schneider, UA; McCarl, BA",Economic potential of biomass based fuels for ...,Use of biofuels diminishes fossil fuel combust...,2003.0,10.1023/A:1023632309097,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,"Step 1: The article's title is ""Economic poten..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1988,J,"Laamrani, A; Voroney, PR; Berg, AA; Gillespie,...",Temporal Change of Soil Carbon on a Long-Term ...,The impacts of tillage practices and crop rota...,2020.0,10.3390/agronomy10060840,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Step 1: Does the article relate to agroecology...
396,1994,J,"Vijayakumar, S; Bazrgar, AB; Coleman, B; Gordo...",Carbon stocks in riparian buffer systems at si...,"In any given watershed, riparian buffer system...",2020.0,10.1016/j.agee.2020.107149,INCLUDE,INCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Step 1: Determine the relevance of the article...
397,1995,J,"Edberg, S; Tigabu, M; Oden, PC",Commercial Eucalyptus Plantations with Taungya...,"The increasing demand for wood, fiber, and pul...",2022.0,10.3390/f13091395,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Step 1: Does the article relate to agroecology...
398,1997,J,"Kollah, B; Dubey, G; Parasai, P; Saha, JK; Gan...",Interactive effect of biochar size and organic...,Biochar (BC) application to agricultural soil ...,2015.0,10.1111/sum.12168,EXCLUDE,NaN,EXCLUDE,EXCLUDE,NaN,NaN,Step 1: Identify the main topic of the article...


In [47]:
test_pred.to_excel("chatgpt_responses_test_pred_cot_s0_v3.xlsx")

In [48]:
from sklearn.metrics import classification_report

In [49]:
print(classification_report(test_pred["Final decision"].apply(lambda x: int("include" in x.lower())), test_pred["chatgpt"].apply(lambda x: int("include" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     0.4459    0.7234    0.5517       188
           1     0.4526    0.2028    0.2801       212

    accuracy                         0.4475       400
   macro avg     0.4493    0.4631    0.4159       400
weighted avg     0.4495    0.4475    0.4078       400

